In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns
import os
import joblib

In [ ]:
# ===== LOAD DATA =====
# Load the dataset
df_raw = pd.read_csv('yield.csv')
df_raw.head()


,Domain Code,Domain,Area Code,Area,Element Code,Element,Item Code,Item,Year Code,Year,Unit,Value
0,QC,Crops,2,Afghanistan,5419,Yield,56,Maize,1961,1961,hg/ha,14000
1,QC,Crops,2,Afghanistan,5419,Yield,56,Maize,1962,1962,hg/ha,14000
2,QC,Crops,2,Afghanistan,5419,Yield,56,Maize,1963,1963,hg/ha,14260
3,QC,Crops,2,Afghanistan,5419,Yield,56,Maize,1964,1964,hg/ha,14257
4,QC,Crops,2,Afghanistan,5419,Yield,56,Maize,1965,1965,hg/ha,14400


In [5]:
df_raw.columns


Index(['Domain Code', 'Domain', 'Area Code', 'Area', 'Element Code', 'Element',
       'Item Code', 'Item', 'Year Code', 'Year', 'Unit', 'Value'],
      dtype='object')

In [ ]:
# Exploring the FAO dataset structure to understand what we're working with
print("\n" + "=" * 70)
print(" DATA STRUCTURE ANALYSIS")
print("=" * 70)

print(f"\n📊 Dataset Overview:")
print(f"   Total records: {len(df_raw):,}")
print(f"   Countries: {df_raw['Area'].nunique()}")
print(f"   Crops: {df_raw['Item'].nunique()}")
print(f"   Years: {df_raw['Year'].min()} to {df_raw['Year'].max()}")
print(f"   Unit: {df_raw['Unit'].unique()[0]}")

print(f"\n🌍 Sample Countries:")
print(f"   {df_raw['Area'].unique()[:10].tolist()}")

print(f"\n🌾 Crop Types:")
print(f"   {df_raw['Item'].unique().tolist()}")

print(f"\n⚠️  Missing Values:")
missing = df_raw.isnull().sum()
missing = missing[missing > 0]
if len(missing) > 0:
    for col, count in missing.items():
        print(f"   {col}: {count:,} ({count/len(df_raw)*100:.2f}%)")
else:
    print("   ✅ No missing values in key columns!")



 DATA STRUCTURE ANALYSIS

📊 Dataset Overview:
   Total records: 56,717
   Countries: 212
   Crops: 10
   Years: 1961 to 2016
   Unit: hg/ha

🌍 Sample Countries:
   ['Afghanistan', 'Albania', 'Algeria', 'American Samoa', 'Angola', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Australia', 'Austria']

🌾 Crop Types:
   ['Maize', 'Potatoes', 'Rice, paddy', 'Wheat', 'Sorghum', 'Soybeans', 'Cassava', 'Yams', 'Sweet potatoes', 'Plantains and others']

⚠️  Missing Values:
   ✅ No missing values in key columns!


In [9]:
df_raw.columns

Index(['Domain Code', 'Domain', 'Area Code', 'Area', 'Element Code', 'Element',
       'Item Code', 'Item', 'Year Code', 'Year', 'Unit', 'Value'],
      dtype='object')

In [10]:
df_raw.head()

,Domain Code,Domain,Area Code,Area,Element Code,Element,Item Code,Item,Year Code,Year,Unit,Value
0,QC,Crops,2,Afghanistan,5419,Yield,56,Maize,1961,1961,hg/ha,14000
1,QC,Crops,2,Afghanistan,5419,Yield,56,Maize,1962,1962,hg/ha,14000
2,QC,Crops,2,Afghanistan,5419,Yield,56,Maize,1963,1963,hg/ha,14260
3,QC,Crops,2,Afghanistan,5419,Yield,56,Maize,1964,1964,hg/ha,14257
4,QC,Crops,2,Afghanistan,5419,Yield,56,Maize,1965,1965,hg/ha,14400


In [17]:
df_raw['Value'].describe()

count      56717.000000
mean       62094.660084
std        67835.932856
min            0.000000
25%        15680.000000
50%        36744.000000
75%        86213.000000
max      1000000.000000
Name: Value, dtype: float64

In [ ]:
print("\n" + "=" * 70)
print("DATA CLEANING AND PREPARATION")
print("=" * 70)

# Keeping only the columns we need for modeling
df = df_raw[['Area', 'Item', 'Year', 'Value']].copy()

# Renaming columns to be more intuitive
df.rename(columns={
    'Area': 'Country',
    'Item': 'Crop',
    'Value': 'Yield_hg_per_ha'
}, inplace=True)

# Converting to tons/hectare makes the results easier to interpret
# Conversion: 1 hg/ha = 0.0001 tons/ha
df['Yield_tons_per_ha'] = df['Yield_hg_per_ha'] * 0.0001

# Filtering out invalid data points
initial_rows = len(df)
df = df[df['Yield_tons_per_ha'] > 0]
df = df[df['Yield_tons_per_ha'] < 100]  # Removing unrealistic outliers
print(f"   Removed {initial_rows - len(df)} rows with invalid yield values")
print(f"   Final dataset: {len(df):,} rows")

print(f"\n📋 Sample data (first 10 rows):")
print(df.head(10).to_string())

print(f"\n📊 Yield Statistics (tons/hectare):")
print(df['Yield_tons_per_ha'].describe())

print(f"\n✅ Data cleaning complete!")



DATA CLEANING AND PREPARATION
   Removed 9 rows with invalid yield values
   Final dataset: 56,708 rows

📋 Sample data (first 10 rows):
       Country   Crop  Year  Yield_hg_per_ha  Yield_tons_per_ha
0  Afghanistan  Maize  1961            14000             1.4000
1  Afghanistan  Maize  1962            14000             1.4000
2  Afghanistan  Maize  1963            14260             1.4260
3  Afghanistan  Maize  1964            14257             1.4257
4  Afghanistan  Maize  1965            14400             1.4400
5  Afghanistan  Maize  1966            14400             1.4400
6  Afghanistan  Maize  1967            14144             1.4144
7  Afghanistan  Maize  1968            17064             1.7064
8  Afghanistan  Maize  1969            17177             1.7177
9  Afghanistan  Maize  1970            14757             1.4757

📊 Yield Statistics (tons/hectare):
count    56708.000000
mean         6.208688
std          6.772287
min          0.005000
25%          1.569075
50%          

In [ ]:
print("\n" + "=" * 70)
print(" FEATURE ENGINEERING")
print("=" * 70)

# One-hot encoding categorical variables so the models can use them
# Country and crop type are important predictors since different regions and crops have varying yield potentials
print("\n🔢 Encoding categorical features...")

df_encoded = pd.get_dummies(df, columns=['Country', 'Crop'], prefix=['Country', 'Crop'])

# Converting to integers ensures compatibility with all ML algorithms
one_hot_cols = [col for col in df_encoded.columns 
                if col.startswith('Country_') or col.startswith('Crop_')]
df_encoded[one_hot_cols] = df_encoded[one_hot_cols].astype(int)

print(f"   ✅ One-hot encoded {df['Country'].nunique()} countries")
print(f"   ✅ One-hot encoded {df['Crop'].nunique()} crop types")
print(f"   ✅ Converted one-hot columns to integers (0/1) for ML")

# Year is already numerical, so I'm keeping it as-is
print(f"   ✅ Year column kept as numerical feature (range: {df['Year'].min()}-{df['Year'].max()})")

# Dropping the original yield column since we're using the converted version
df_encoded = df_encoded.drop(columns=['Yield_hg_per_ha'], errors='ignore')
print(f"   ✅ Removed original yield column")

# Building the feature matrix
feature_cols = [col for col in df_encoded.columns 
                if col.startswith('Country_') or 
                   col.startswith('Crop_') or 
                   col == 'Year']

X = df_encoded[feature_cols].values
y = df_encoded['Yield_tons_per_ha'].values

print(f"\n✅ Features created!")
print(f"   Total features: {len(feature_cols)}")
print(f"   - One-hot encoded countries: {sum(1 for col in feature_cols if col.startswith('Country_'))}")
print(f"   - One-hot encoded crops: {sum(1 for col in feature_cols if col.startswith('Crop_'))}")
print(f"   - Year: 1")
print(f"   Target: Yield (tons/hectare)")
print(f"   Samples: {len(df_encoded):,}")

# Keeping only what we need for modeling
final_cols = feature_cols + ['Yield_tons_per_ha']
df_encoded = df_encoded[final_cols]
print(f"\n🧹 Final cleanup complete")
print(f"   ✅ Final dataframe columns: {len(df_encoded.columns)}")

print(f"\n📊 Feature Statistics:")
print(f"Year column statistics:")
print(df_encoded['Year'].describe())
print(f"\nSample of one-hot encoded columns:")
sample_cols = [col for col in feature_cols if col.startswith('Country_')][:5] + \
              [col for col in feature_cols if col.startswith('Crop_')][:5]
print(df_encoded[sample_cols].head())


 FEATURE ENGINEERING

🔢 Encoding categorical features...
   ✅ One-hot encoded 212 countries
   ✅ One-hot encoded 10 crop types
   ✅ Original Country and Crop columns dropped
   ✅ Converted one-hot columns to integers (0/1) for ML
   ✅ Year column kept as numerical feature (range: 1961-2016)
   ✅ Removed unwanted columns: ['Yield_hg_per_ha']

✅ Features created!
   Total features: 223
   - One-hot encoded countries: 212
   - One-hot encoded crops: 10
   - Year: 1
   Target: Yield (tons/hectare)
   Samples: 56,708

🧹 Final cleanup - keeping only features and target...
   ✅ Final dataframe columns: 224
      - Features: 223
      - Target: 1 (Yield_tons_per_ha)

📊 Feature Statistics:
Year column statistics:
count    56708.000000
mean      1989.667913
std         16.133039
min       1961.000000
25%       1976.000000
50%       1991.000000
75%       2004.000000
max       2016.000000
Name: Year, dtype: float64

Sample of one-hot encoded columns (showing as integers 0/1):
   Country_Afghanist

In [28]:
df_encoded

,Year,Country_Afghanistan,Country_Albania,Country_Algeria,Country_American Samoa,Country_Angola,Country_Antigua and Barbuda,Country_Argentina,Country_Armenia,Country_Australia,...,Crop_Maize,Crop_Plantains and others,Crop_Potatoes,"Crop_Rice, paddy",Crop_Sorghum,Crop_Soybeans,Crop_Sweet potatoes,Crop_Wheat,Crop_Yams,Yield_tons_per_ha
0,1961,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1.4000
1,1962,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1.4000
2,1963,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1.4260
3,1964,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1.4257
4,1965,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1.4400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56712,2012,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,2.4420
56713,2013,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,2.2888
56714,2014,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,2.1357
56715,2015,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1.9826


In [ ]:
print("\n" + "=" * 70)
print(" DATA PREPROCESSING")
print("=" * 70)

# Splitting before scaling prevents data leakage - the test set shouldn't influence any preprocessing steps
X = df_encoded[feature_cols].values
y = df_encoded['Yield_tons_per_ha'].values

print("\n📊 Splitting data into train/test sets...")
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    shuffle=True
)

print(f"   ✅ Training set: {X_train.shape[0]:,} samples ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"   ✅ Test set: {X_test.shape[0]:,} samples ({X_test.shape[0]/len(X)*100:.1f}%)")
print(f"   ✅ Features: {X_train.shape[1]}")

# Scaling is important because Year has a much larger scale than the binary one-hot features
# This helps algorithms like neural networks and linear regression converge better
print("\n📏 Scaling features...")
scaler = StandardScaler()

# Fitting the scaler only on training data, then applying to both sets
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("   ✅ Features scaled using StandardScaler (fitted on train only)")
print(f"      Train range: [{X_train_scaled.min():.2f}, {X_train_scaled.max():.2f}]")
print(f"      Test range: [{X_test_scaled.min():.2f}, {X_test_scaled.max():.2f}]")

print("\n✅ Data preprocessing complete (no data leakage!)")



 DATA PREPROCESSING

📊 Splitting data into train/test sets...
   ✅ Training set: 45,366 samples (80.0%)
   ✅ Test set: 11,342 samples (20.0%)
   ✅ Features: 223

📏 Scaling features...
   ✅ Features scaled using StandardScaler (fitted on train only)
      Train range: [-1.78, 54.99]
      Test range: [-1.78, 54.99]

✅ Data preprocessing complete (no data leakage!)


In [ ]:
print("\n" + "=" * 70)
print("MODEL TRAINING AND COMPARISON")
print("=" * 70)

# Comparing multiple algorithms to find the best approach
models = {}
results = {}

os.makedirs('outputs', exist_ok=True)


MODEL TRAINING AND COMPARISON


In [ ]:
print("\n" + "-" * 70)
print(" LINEAR REGRESSION (Baseline)")
print("-" * 70)

# Starting with a simple linear model as a baseline for comparison
lr_model = LinearRegression()
print("   Training Linear Regression...")
lr_model.fit(X_train_scaled, y_train)

y_pred_lr = lr_model.predict(X_test_scaled)

mse_lr = mean_squared_error(y_test, y_pred_lr)
rmse_lr = np.sqrt(mse_lr)
mae_lr = mean_absolute_error(y_test, y_pred_lr)
r2_lr = r2_score(y_test, y_pred_lr)

models['Linear Regression'] = lr_model
results['Linear Regression'] = {
    'RMSE': rmse_lr,
    'MAE': mae_lr,
    'R2': r2_lr,
    'predictions': y_pred_lr
}

print(f"   ✅ Training complete!")
print(f"   📊 Results:")
print(f"      RMSE: {rmse_lr:.4f} tons/hectare")
print(f"      MAE:  {mae_lr:.4f} tons/hectare")
print(f"      R²:   {r2_lr:.4f}")



----------------------------------------------------------------------
 LINEAR REGRESSION (Baseline)
----------------------------------------------------------------------
   Training Linear Regression...
   ✅ Training complete!
   📊 Results:
      RMSE: 3.9087 tons/hectare
      MAE:  2.6637 tons/hectare
      R²:   0.6633


In [38]:
#  RANDOM FOREST REGRESSION
# ============================================================================

print("\n" + "-" * 70)
print(" RANDOM FOREST REGRESSION")
print("-" * 70)

rf_model = RandomForestRegressor(
    n_estimators=100,
    max_depth=15,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42,
    n_jobs=-1,
    verbose=0
)

print("   Training Random Forest (this may take a few minutes)...")
rf_model.fit(X_train_scaled, y_train)

y_pred_rf = rf_model.predict(X_test_scaled)

mse_rf = mean_squared_error(y_test, y_pred_rf)
rmse_rf = np.sqrt(mse_rf)
mae_rf = mean_absolute_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)

models['Random Forest'] = rf_model
results['Random Forest'] = {
    'RMSE': rmse_rf,
    'MAE': mae_rf,
    'R2': r2_rf,
    'predictions': y_pred_rf
}

print(f"   ✅ Training complete!")
print(f"   📊 Results:")
print(f"      RMSE: {rmse_rf:.4f} tons/hectare")
print(f"      MAE:  {mae_rf:.4f} tons/hectare")
print(f"      R²:   {r2_rf:.4f}")

# Feature importance
feature_importance = pd.DataFrame({
    'Feature': feature_cols,
    'Importance': rf_model.feature_importances_
}).sort_values('Importance', ascending=False)

print(f"\n   🔍 Top 5 Most Important Features:")
for idx, row in feature_importance.head(5).iterrows():
    print(f"      {row['Feature']:25s}: {row['Importance']:.4f}")


----------------------------------------------------------------------
 RANDOM FOREST REGRESSION
----------------------------------------------------------------------
   Training Random Forest (this may take a few minutes)...
   ✅ Training complete!
   📊 Results:
      RMSE: 3.4470 tons/hectare
      MAE:  2.3001 tons/hectare
      R²:   0.7381

   🔍 Top 5 Most Important Features:
      Crop_Potatoes            : 0.3687
      Crop_Sweet potatoes      : 0.0814
      Crop_Cassava             : 0.0741
      Year                     : 0.0735
      Crop_Plantains and others: 0.0447


In [39]:
#  XGBOOST REGRESSION
# ============================================================================

print("\n" + "-" * 70)
print(" XGBOOST REGRESSION")
print("-" * 70)

xgb_model = xgb.XGBRegressor(
    n_estimators=200,
    max_depth=8,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1,
    verbosity=0
)

print("   Training XGBoost (this may take a few minutes)...")
xgb_model.fit(X_train_scaled, y_train)

y_pred_xgb = xgb_model.predict(X_test_scaled)

mse_xgb = mean_squared_error(y_test, y_pred_xgb)
rmse_xgb = np.sqrt(mse_xgb)
mae_xgb = mean_absolute_error(y_test, y_pred_xgb)
r2_xgb = r2_score(y_test, y_pred_xgb)

models['XGBoost'] = xgb_model
results['XGBoost'] = {
    'RMSE': rmse_xgb,
    'MAE': mae_xgb,
    'R2': r2_xgb,
    'predictions': y_pred_xgb
}

print(f"   ✅ Training complete!")
print(f"   📊 Results:")
print(f"      RMSE: {rmse_xgb:.4f} tons/hectare")
print(f"      MAE:  {mae_xgb:.4f} tons/hectare")
print(f"      R²:   {r2_xgb:.4f}")


----------------------------------------------------------------------
 XGBOOST REGRESSION
----------------------------------------------------------------------
   Training XGBoost (this may take a few minutes)...
   ✅ Training complete!
   📊 Results:
      RMSE: 2.9697 tons/hectare
      MAE:  2.0001 tons/hectare
      R²:   0.8056


In [ ]:
print("\n" + "-" * 70)
print(" NEURAL NETWORK REGRESSION")
print("-" * 70)

from sklearn.neural_network import MLPRegressor

# Neural networks can capture complex non-linear relationships that simpler models might miss
# I'm using a multi-layer architecture with decreasing neuron counts to learn hierarchical patterns
nn_model = MLPRegressor(
    hidden_layer_sizes=(128, 64, 32),
    activation='relu',
    solver='adam',
    alpha=0.001,
    learning_rate='adaptive',
    max_iter=500,
    early_stopping=True,
    validation_fraction=0.1,
    n_iter_no_change=20,
    random_state=42,
    verbose=False
)

print("   Training Neural Network (this may take a few minutes)...")
print("   Architecture: Input → 128 → 64 → 32 → Output")
print("   Using early stopping to prevent overfitting...")

nn_model.fit(X_train_scaled, y_train)

y_pred_nn = nn_model.predict(X_test_scaled)

mse_nn = mean_squared_error(y_test, y_pred_nn)
rmse_nn = np.sqrt(mse_nn)
mae_nn = mean_absolute_error(y_test, y_pred_nn)
r2_nn = r2_score(y_test, y_pred_nn)

models['Neural Network'] = nn_model
results['Neural Network'] = {
    'RMSE': rmse_nn,
    'MAE': mae_nn,
    'R2': r2_nn,
    'predictions': y_pred_nn
}

print(f"   ✅ Training complete!")
print(f"   📊 Results:")
print(f"      RMSE: {rmse_nn:.4f} tons/hectare")
print(f"      MAE:  {mae_nn:.4f} tons/hectare")
print(f"      R²:   {r2_nn:.4f}")
print(f"   Final iterations: {nn_model.n_iter_}")


----------------------------------------------------------------------
 NEURAL NETWORK REGRESSION
----------------------------------------------------------------------
   Training Neural Network (this may take a few minutes)...
   Architecture: Input → 128 → 64 → 32 → Output
   Using early stopping to prevent overfitting...
   ✅ Training complete!
   📊 Results:
      RMSE: 1.2977 tons/hectare
      MAE:  0.6709 tons/hectare
      R²:   0.9629
   Final iterations: 114


In [ ]:
print("\n" + "=" * 70)
print(" MODEL COMPARISON")
print("=" * 70)

comparison_df = pd.DataFrame({
    'Model': list(results.keys()),
    'RMSE': [results[m]['RMSE'] for m in results.keys()],
    'MAE': [results[m]['MAE'] for m in results.keys()],
    'R2': [results[m]['R2'] for m in results.keys()]
})

comparison_df = comparison_df.sort_values('R2', ascending=False)

print("\n📊 Model Performance Comparison:")
print(comparison_df.to_string(index=False))

# Creating visualizations to compare model performance across different metrics
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

metrics = ['RMSE', 'MAE', 'R2']
colors = ['#2ecc71', '#3498db', '#e74c3c', '#9b59b6']
for i, metric in enumerate(metrics):
    axes[i].barh(comparison_df['Model'], comparison_df[metric], 
                 color=colors[:len(comparison_df)])
    axes[i].set_xlabel(metric, fontsize=12)
    axes[i].set_title(f'Model Comparison - {metric}', fontsize=14, fontweight='bold')
    axes[i].grid(True, alpha=0.3, axis='x')
    
    for j, v in enumerate(comparison_df[metric]):
        axes[i].text(v, j, f' {v:.4f}', va='center', fontsize=10)
    
    if metric == 'R2':
        axes[i].set_xlim([0, max(1.0, comparison_df[metric].max() * 1.1)])

plt.tight_layout()
plt.savefig('outputs/crop_yield_model_comparison.png', dpi=150, bbox_inches='tight')
plt.close()
print("\n✅ Saved: outputs/crop_yield_model_comparison.png")

# Selecting the best performing model based on R² score
best_model_name = comparison_df.iloc[0]['Model']
best_model = models[best_model_name]
best_r2 = comparison_df.iloc[0]['R2']

print(f"\n🏆 Best Model: {best_model_name}")
print(f"   R² Score: {best_r2:.4f}")
print(f"   RMSE: {comparison_df.iloc[0]['RMSE']:.4f} tons/hectare")


 MODEL COMPARISON

📊 Model Performance Comparison:
            Model     RMSE      MAE       R2
   Neural Network 1.297739 0.670873 0.962886
          XGBoost 2.969724 2.000064 0.805646
    Random Forest 3.447047 2.300083 0.738148
Linear Regression 3.908657 2.663654 0.663321

✅ Saved: outputs/crop_yield_model_comparison.png

🏆 Best Model: Neural Network
   R² Score: 0.9629
   RMSE: 1.2977 tons/hectare


In [43]:
#  VISUALIZATIONS
# ============================================================================

print("\n" + "=" * 70)
print(" VISUALIZATIONS")
print("=" * 70)

y_pred_best = best_model.predict(X_test_scaled)

# 1. Predictions vs Actual
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.scatter(y_test, y_pred_best, alpha=0.5, s=20)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2, label='Perfect Prediction')
plt.xlabel('Actual Yield (tons/hectare)', fontsize=12)
plt.ylabel('Predicted Yield (tons/hectare)', fontsize=12)
plt.title(f'Predictions vs Actual - {best_model_name}', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.text(0.95, 0.95, f'R² = {best_r2:.4f}', transform=plt.gca().transAxes,
         fontsize=12, verticalalignment='top', horizontalalignment='right',
         bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

# 2. Feature importance (if Random Forest or XGBoost)
if best_model_name in ['Random Forest', 'XGBoost']:
    plt.subplot(1, 2, 2)
    top_features = feature_importance.head(10)
    plt.barh(range(len(top_features)), top_features['Importance'].values)
    plt.yticks(range(len(top_features)), top_features['Feature'].values)
    plt.xlabel('Feature Importance', fontsize=12)
    plt.title('Top 10 Feature Importance', fontsize=14, fontweight='bold')
    plt.gca().invert_yaxis()
    plt.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.savefig('outputs/crop_yield_predictions.png', dpi=150, bbox_inches='tight')
plt.close()
print("✅ Saved: outputs/crop_yield_predictions.png")

# 3. Yield by crop type
plt.figure(figsize=(14, 6))
crop_yield = df.groupby('Crop')['Yield_tons_per_ha'].mean().sort_values(ascending=False)
plt.barh(range(len(crop_yield)), crop_yield.values)
plt.yticks(range(len(crop_yield)), crop_yield.index)
plt.xlabel('Average Yield (tons/hectare)', fontsize=12)
plt.ylabel('Crop Type', fontsize=12)
plt.title('Average Yield by Crop Type', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.grid(True, alpha=0.3, axis='x')
plt.tight_layout()
plt.savefig('outputs/crop_yield_by_crop.png', dpi=150, bbox_inches='tight')
plt.close()
print("✅ Saved: outputs/crop_yield_by_crop.png")

# 4. Yield trends over time
plt.figure(figsize=(14, 6))
yearly_yield = df.groupby('Year')['Yield_tons_per_ha'].mean()
plt.plot(yearly_yield.index, yearly_yield.values, marker='o', linewidth=2, markersize=4)
plt.xlabel('Year', fontsize=12)
plt.ylabel('Average Yield (tons/hectare)', fontsize=12)
plt.title('Global Crop Yield Trends Over Time (1961-2016)', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('outputs/crop_yield_trends.png', dpi=150, bbox_inches='tight')
plt.close()
print("✅ Saved: outputs/crop_yield_trends.png")

# 


 VISUALIZATIONS
✅ Saved: outputs/crop_yield_predictions.png
✅ Saved: outputs/crop_yield_by_crop.png
✅ Saved: outputs/crop_yield_trends.png


In [54]:
#  SAVE MODEL
# ============================================================================

print("\n" + "=" * 70)
print(" SAVE MODEL")
print("=" * 70)

os.makedirs('models', exist_ok=True)

model_filename = best_model_name.lower().replace(" ", "_")
model_path = f'models/{model_filename}_crop_yield.pkl'
scaler_path = 'models/crop_yield_scaler.pkl'
feature_cols_path = 'models/crop_yield_feature_columns.pkl'

# Save the best model (trained on scaled data)
joblib.dump(best_model, model_path)
print(f"✅ Model saved to: {model_path}")

# Save the scaler (fitted on training data)
joblib.dump(scaler, scaler_path)
print(f"✅ Scaler saved to: {scaler_path}")

# Save feature column names in the EXACT order used during training
# This is critical: the model expects features in this specific order
# During inference, we'll use this list to reorder columns to match training data
joblib.dump(feature_cols, feature_cols_path)
print(f"✅ Feature columns saved to: {feature_cols_path}")
print(f"   Total features: {len(feature_cols)}")
print(f"   - Countries: {sum(1 for col in feature_cols if col.startswith('Country_'))}")
print(f"   - Crops: {sum(1 for col in feature_cols if col.startswith('Crop_'))}")
print(f"   ⚠️  Column order is preserved - critical for model predictions!")


# 


 SAVE MODEL
✅ Model saved to: models/neural_network_crop_yield.pkl
✅ Scaler saved to: models/crop_yield_scaler.pkl
✅ Feature columns saved to: models/crop_yield_feature_columns.pkl
   Total features: 223
   - Countries: 212
   - Crops: 10
   ⚠️  Column order is preserved - critical for model predictions!


In [ ]:
def predict_crop_yield(country, crop, year, 
                       model_path=None, 
                       scaler_path='models/crop_yield_scaler.pkl',
                       feature_cols_path='models/crop_yield_feature_columns.pkl'):
    """
    Predict crop yield for given country, crop, and year.
    
    Parameters:
    -----------
    country : str
        Country name (e.g., 'United States', 'India')
    crop : str
        Crop type (e.g., 'Maize', 'Wheat', 'Rice, paddy')
    year : int
        Year (should be between 1961-2016 for training data range)
    model_path : str, optional
        Path to saved model. If None, will try to find best model.
    scaler_path : str
        Path to saved scaler
    feature_cols_path : str
        Path to saved feature columns
    
    Returns:
    --------
    float
        Predicted yield in tons per hectare
    """
    
    print("Loading saved model components...")
    
    feature_cols = joblib.load(feature_cols_path)
    print(f"   ✅ Loaded {len(feature_cols)} feature columns")
    
    loaded_scaler = joblib.load(scaler_path)
    print(f"   ✅ Loaded scaler")
    
    if model_path is None:
        model_files = [f for f in os.listdir('models') if f.endswith('_crop_yield.pkl')]
        if not model_files:
            raise FileNotFoundError("No model file found. Please specify model_path.")
        model_path = f'models/{model_files[0]}'
    
    loaded_model = joblib.load(model_path)
    print(f"   ✅ Loaded model from: {model_path}")
    
    print(f"\nMaking prediction for:")
    print(f"   Country: {country}")
    print(f"   Crop: {crop}")
    print(f"   Year: {year}")
    
    # Replicating the same preprocessing steps used during training
    input_df = pd.DataFrame({
        'Country': [country],
        'Crop': [crop],
        'Year': [year]
    })
    
    input_encoded = pd.get_dummies(input_df, columns=['Country', 'Crop'], 
                                   prefix=['Country', 'Crop'])
    
    one_hot_cols = [col for col in input_encoded.columns 
                    if col.startswith('Country_') or col.startswith('Crop_')]
    if one_hot_cols:
        input_encoded[one_hot_cols] = input_encoded[one_hot_cols].astype(int)
    
    # Adding missing feature columns (set to 0) to match training structure
    missing_cols = [col for col in feature_cols if col not in input_encoded.columns]
    if missing_cols:
        missing_df = pd.DataFrame(0, index=input_encoded.index, columns=missing_cols)
        input_encoded = pd.concat([input_encoded, missing_df], axis=1)
    
    # Ensuring features are in the exact same order as during training
    X_input = input_encoded[feature_cols].values
    
    if list(input_encoded[feature_cols].columns) != feature_cols:
        raise ValueError("Feature column order mismatch! This should never happen.")
    
    print(f"   ✅ Feature matrix created with {X_input.shape[1]} features in correct order")
    
    X_input_scaled = loaded_scaler.transform(X_input)
    prediction = loaded_model.predict(X_input_scaled)[0]
    
    print(f"\n✅ Predicted Yield: {prediction:.4f} tons/hectare")
    
    return prediction


# Example usage:
prediction = predict_crop_yield('United States', 'Maize', 2020)
print(f"Predicted yield: {prediction:.2f} tons/hectare")


Loading saved model components...
   ✅ Loaded 223 feature columns
   ✅ Loaded scaler
   ✅ Loaded model from: models/neural_network_crop_yield.pkl

Making prediction for:
   Country: United States
   Crop: Maize
   Year: 2020
   ✅ Feature matrix created with 223 features in correct order

✅ Predicted Yield: 3.1593 tons/hectare
Predicted yield: 3.16 tons/hectare


In [ ]:
print("\n" + "=" * 70)
print("PROJECT 4 COMPLETE! 🎉")
print("=" * 70)

print(f"\n📊 Final Results:")
print(f"   Best Model: {best_model_name}")
print(f"   R² Score: {results[best_model_name]['R2']:.4f}")
print(f"   RMSE: {results[best_model_name]['RMSE']:.4f} tons/hectare")
print(f"   MAE: {results[best_model_name]['MAE']:.4f} tons/hectare")

print(f"\n📁 Outputs saved:")
print(f"   - Model: {model_path}")
print(f"   - Visualizations: outputs/ directory")

print(f"\n💡 Key Insights:")
print(f"   - Dataset: {len(df):,} records from {df['Country'].nunique()} countries")
print(f"   - Crops: {df['Crop'].nunique()} different crop types")
print(f"   - Time period: {df['Year'].min()} to {df['Year'].max()}")
print(f"   - Average yield: {df['Yield_tons_per_ha'].mean():.2f} tons/hectare")

print("\n" + "=" * 70)


PROJECT 4 COMPLETE! 🎉

📊 Final Results:
   Best Model: Neural Network
   R² Score: 0.9629
   RMSE: 1.2977 tons/hectare
   MAE: 0.6709 tons/hectare

📁 Outputs saved:
   - Model: models/neural_network_crop_yield.pkl
   - Visualizations: outputs/ directory

💡 Key Insights:
   - Dataset: 56,708 records from 212 countries
   - Crops: 10 different crop types
   - Time period: 1961 to 2016
   - Average yield: 6.21 tons/hectare

